## Pair Programming Limpieza II
### Modulo 2 Sprint 2
#### Lola Rubio y Judith Mellidez 

Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. **Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

In [1]:
import pandas as pd
import re
import numpy as np
import re

In [2]:
df=pd.read_csv('attacks_limpieza1.csv', index_col=0)
df.head()

,year,type,country,age,species_,month,fatal,sex
case_number,,,,,,,,
1800.00.00,1800,Unprovoked,seychelles,NaN,NaN,NaN,Y,F
1797.05.28.R,1797,Unprovoked,NaN,NaN,NaN,May,Y,NaN
1792.09.12,1792,Provoked,england,NaN,NaN,Sep,Y,M
1791.00.00,1791,Unprovoked,australia,NaN,NaN,NaN,Y,F
1788.05.10,1788,Boat,australia,NaN,NaN,May,N,NaN


Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

#### 1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [4]:
df['species_'].value_counts()

White shark                                         76
Tiger shark                                         37
4' shark                                            34
Bull shark                                          32
Shark involvement not confirmed                     26
                                                    ..
14' to 16' white shark                               1
juvenile white shark                                 1
Possibly a sandbar shark or small blacktip shark     1
[4.5' to 5'] shark                                   1
Identified as C. gangeticus by Dr. J. Fayrer         1
Name: species_, Length: 856, dtype: int64

In [5]:
#Definimos los patrones antes de introducirlos en la funcion.
patron_white = r".*[Ww](hite|HITE).*"
patron_tiger = r".*[Tt](iger|IGER).*"
patron_grey = r".*[Gg](rey|REY).*"
patron_lemon= r".*[Ll](emon|EMON).*"
patron_bull= r".*[Bb](ull|ULL).*"

In [6]:
def species_shark (x):
    patron_white = re.findall(".*[Ww](hite|HITE).*",str(x))
    patron_tiger = re.findall(".*[Tt](iger|IGER).*",str(x))
    patron_grey = re.findall(".*[Gg](rey|REY).*", str(x))
    patron_lemon= re.findall(".*[Ll](emon|EMON).*", str(x))
    patron_bull= re.findall(".*[Bb](ull|ULL).*", str(x))

    if patron_white:
        return('White')
    elif patron_tiger:
        return('Tiger')
    elif patron_grey:
        return('Grey')
    elif patron_lemon:
        return('Lemon')   
    elif patron_bull:
        return('Bull')     
    else:
        return('Unspecified')

df['species_shark']=df['species_'].apply(species_shark)
     

In [7]:
df.head(8)

,year,type,country,age,species_,month,fatal,sex,species_shark
case_number,,,,,,,,,
1800.00.00,1800,Unprovoked,seychelles,NaN,NaN,NaN,Y,F,Unspecified
1797.05.28.R,1797,Unprovoked,NaN,NaN,NaN,May,Y,NaN,Unspecified
1792.09.12,1792,Provoked,england,NaN,NaN,Sep,Y,M,Unspecified
1791.00.00,1791,Unprovoked,australia,NaN,NaN,NaN,Y,F,Unspecified
1788.05.10,1788,Boat,australia,NaN,NaN,May,N,NaN,Unspecified
1787.07.05,1787,Unprovoked,NaN,NaN,NaN,Jul,Y,M,Unspecified
1786.08.10.R,1786,Unprovoked,barbados,NaN,NaN,Aug,Y,M,Unspecified
1785.09.26.R,1785,Unprovoked,england,NaN,Tiger shark?,Sep,Y,M,Tiger


In [8]:
#comprobamos que se ha realizado bien el cambio 
df['species_shark'].value_counts()

Unspecified    984
White          386
Tiger          157
Bull           104
Grey            30
Lemon           11
Name: species_shark, dtype: int64

In [4]:
df.columns

Index(['year', 'type', 'country', 'age', 'species_', 'month', 'fatal', 'sex'], dtype='object')

#### 2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

- Edad en formato string

- Edades separadas por &, or, to, >

- Edades con ?

In [11]:
df['age'].unique()

array([nan, 'young', '14', '19', '20', '15', '21', '27', '36', '25', '6',
       '16', '50', '13 or 14', '57', '18', '32', '30', '60', '33', '10',
       '69', '55', '35', '54', '22', '31', '40', '37', '11', '13', '34',
       '46', '48', '17', '28', '65', '73', '58', '60s', '51', '61', '59',
       '42', '23', '29', '39', '24', '12', '26', '71', '43', '9', '44',
       '62', '52', '38', '68', '47', '63', '70', '41', '40s', '53', '20s',
       '7', '66', '45', '74', '64', '8', '56', '49', '18 or 20', 'Teen',
       '30s', '77', '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '5',
       ' 30', ' 28', "60's", '67', '>50', '? & 19', '21, 34,24 & 35',
       '30 & 32', '13 or 18', '7 or 8', '9 or 10'], dtype=object)

In [12]:
#Creamos una funcion deonde definimos un patron regex  
def age_clean (x):
    #patron = r'.*\d.*'
    patron= r'\d+\b'
    try:
        age_clean = re.findall(r'\d+\b', x)
        return age_clean
        #return (re.findall('.*\d.*', x))
    except:
        return np.nan
        

In [13]:
df['age_clean']=df['age'].apply(age_clean)

In [14]:
df['age_clean'].value_counts()

[19]                71
[18]                60
[20]                58
[15]                57
[17]                57
                    ..
[21, 34, 24, 35]     1
[30, 32]             1
[13, 18]             1
[7, 8]               1
[9, 10]              1
Name: age_clean, Length: 81, dtype: int64

In [15]:
df2 = df.explode('age_clean', ignore_index=False)


In [16]:
df2

,year,type,country,age,species_,month,fatal,sex,species_shark,age_clean
case_number,,,,,,,,,,
1800.00.00,1800,Unprovoked,seychelles,NaN,NaN,NaN,Y,F,Unspecified,NaN
1797.05.28.R,1797,Unprovoked,NaN,NaN,NaN,May,Y,NaN,Unspecified,NaN
1792.09.12,1792,Provoked,england,NaN,NaN,Sep,Y,M,Unspecified,NaN
1791.00.00,1791,Unprovoked,australia,NaN,NaN,NaN,Y,F,Unspecified,NaN
1788.05.10,1788,Boat,australia,NaN,NaN,May,N,NaN,Unspecified,NaN
...,...,...,...,...,...,...,...,...,...,...
1901.07.30,1901,Unprovoked,south africa,29,White shark,Jul,Y,M,White,29
1894.11.28,1894,Unprovoked,australia,19,10' to 12' shark,Nov,N,M,Unspecified,19
1892.03.02,1892,Provoked,australia,21,12' shark,Mar,N,M,Unspecified,21


In [17]:
df2['age_clean'] = df2['age_clean'].astype('int64', errors = 'ignore')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1683 entries, 1800.00.00 to 1868.05.13
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           1683 non-null   int64 
 1   type           1683 non-null   object
 2   country        1673 non-null   object
 3   age            1529 non-null   object
 4   species_       1557 non-null   object
 5   month          1527 non-null   object
 6   fatal          1583 non-null   object
 7   sex            1668 non-null   object
 8   species_shark  1683 non-null   object
 9   age_clean      1518 non-null   object
dtypes: int64(1), object(9)
memory usage: 144.6+ KB


In [18]:
df2['age_clean']= pd.to_numeric(df2['age_clean'], errors='coerce')

In [19]:
df.explode('age_clean', ignore_index=False)

,year,type,country,age,species_,month,fatal,sex,species_shark,age_clean
case_number,,,,,,,,,,
1800.00.00,1800,Unprovoked,seychelles,NaN,NaN,NaN,Y,F,Unspecified,NaN
1797.05.28.R,1797,Unprovoked,NaN,NaN,NaN,May,Y,NaN,Unspecified,NaN
1792.09.12,1792,Provoked,england,NaN,NaN,Sep,Y,M,Unspecified,NaN
1791.00.00,1791,Unprovoked,australia,NaN,NaN,NaN,Y,F,Unspecified,NaN
1788.05.10,1788,Boat,australia,NaN,NaN,May,N,NaN,Unspecified,NaN
...,...,...,...,...,...,...,...,...,...,...
1901.07.30,1901,Unprovoked,south africa,29,White shark,Jul,Y,M,White,29
1894.11.28,1894,Unprovoked,australia,19,10' to 12' shark,Nov,N,M,Unspecified,19
1892.03.02,1892,Provoked,australia,21,12' shark,Mar,N,M,Unspecified,21


In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1683 entries, 1800.00.00 to 1868.05.13
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           1683 non-null   int64  
 1   type           1683 non-null   object 
 2   country        1673 non-null   object 
 3   age            1529 non-null   object 
 4   species_       1557 non-null   object 
 5   month          1527 non-null   object 
 6   fatal          1583 non-null   object 
 7   sex            1668 non-null   object 
 8   species_shark  1683 non-null   object 
 9   age_clean      1518 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 144.6+ KB


Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:

 - Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
- Puede que os salga un error similar a este:
TypeError: expected string or bytes-like object
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
df['nombre_columna'] = df['nombre_columna'].astype(str)
- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.
- Por último cambiad el tipo de la columna de string a integer.

### 3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [22]:
df.to_csv('attacks_limpieza2.csv')